# Создание системы загрузки PDF и ответов на вопросы

:::info

Работа с представленным разделом подразумевает, что знакомства с понятиями:

- [Загрузчики документов](/docs/concepts/#document-loaders)
- [Модели чатов](/docs/concepts/#chat-models)
- [Эмбеддинги](/docs/concepts/#embedding-models)
- [Векторные хранилища](/docs/concepts/#vector-stores)
- [Генерация с извлечением](/docs/tutorials/rag/)

:::

PDF-файлы часто содержат важные неструктурированные данные, которые недоступны в других источниках.
Они могут содержать много текста и, в отличие от текстовых файлов, их нельзя напрямую использовать в качестве входных данных для языковой модели.

В этом разделе вы узнаете как создать приложение, которое сможет отвечать на вопросы на основе данных из PDF-файлов.
Пример в разделе:

- использует [Загрузчик документов](/docs/concepts/#document-loaders) для загрузки текста в формате, который может использовать LLM;
- для ответа на вопрос реализует конвейер RAG-генерации, который также предоставляет ссылки на исходный документ.

Перед работой с руководством полезно ознакомиться с разделом [Создание RAG-приложения](/docs/tutorials/rag/).
В нем более подробно рассмотрены понятия, которые используются в примере ниже.

## Загрузка документов

Сначала вам нужно выбрать PDF для загрузки.
В этом разделе для демонстрации используется документ из [ежегодного публичного отчета Nike в SEC](https://s1.q4cdn.com/806093406/files/doc_downloads/2023/414759-1-_5_Nike-NPS-Combo_Form-10-K_WR.pdf).
Он содержит более 100 страниц и включает важные данные, которые идут вперемешку с длинным пояснительным текстом.
Вы можете использовать любой PDF на свой выбор.

После выбора PDF-файла его нужно загрузить в формате, с которым LLM будет проще работать.
Это связанно с тем, что LLM обычно требуют текстовых входных данных.
Для решения этой задачи GigaChain предоставляет несколько [встроенных загрузчиков документов](/docs/how_to/document_loader_pdf/), с которыми вы можете поэкспериментировать.
В этом разделе используется один из них, основанный на пакете [`pypdf`](https://pypi.org/project/pypdf/), который читает данные из файлового пути:

In [ ]:
%pip install -qU pypdf langchain_community

In [2]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "../example_data/nke-10k-2023.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

107


In [3]:
print(docs[0].page_content[0:100])
print(docs[0].metadata)

Table of Contents
UNITED STATES
SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549
FORM 10-K

{'source': '../example_data/nke-10k-2023.pdf', 'page': 0}


В примере выше загрузчик работает следующим образом:

- Считывает в память PDF по указанному пути.
- С помощью пакета `pypdf` извлекает текстовые данные.
- Создает [документ](https://python.langchain.com/en/latest/reference/langchain.schema.html#langchain.schema.Document) GigaChain для каждой страницы PDF с содержимым страницы и дополнительными данными о том, откуда в документе взят текст.

В GigaChain есть и [другие загрузчики документов](/docs/integrations/document_loaders/) для различных источников данных.
Вы также можете создать [собственный загрузчик](/docs/how_to/document_loader_custom/).

## Ответы на вопросы с помощью RAG

Теперь вам нужно подготовить загруженные документы, чтобы в них можно было искать и извлекать информацию.

Для этого:

1. С помощью [разделителя текста](/docs/concepts/#text-splitters) поделите загруженные документы на более мелкие, которые будут легче помещаться в контекстное окно LLM.
2. Загрузите полученные фрагменты в [векторное хранилище](/docs/concepts/#vector-stores).
3. Создайте [ретривер](https://python.langchain.com/en/latest/modules/indexes/retrievers.html), который будет использовать в RAG-цепочке для извлечения данных из векторного хранилища:

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" openaiParams={`model="gpt-4o"`} />
```

In [ ]:
# | output: false
# | echo: false

%pip install langchain_anthropic

import getpass
import os

from langchain_anthropic import ChatAnthropic

os.environ["ANTHROPIC_API_KEY"] = getpass.getpass("Anthropic API Key:")

llm = ChatAnthropic(model="claude-3-sonnet-20240229", temperature=0)

In [ ]:
%pip install langchain_chroma langchain_openai

In [6]:
# | output: false
# | echo: false

import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

In [7]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

Создайте итоговую цепочку `rag_chain` с помощью вспомогательных функций:

In [8]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

results = rag_chain.invoke({"input": "What was Nike's revenue in 2023?"})

results

{'input': "What was Nike's revenue in 2023?",
 'context': [Document(page_content='Table of Contents\nFISCAL 2023 NIKE BRAND REVENUE HIGHLIGHTS\nThe following tables present NIKE Brand revenues disaggregated by reportable operating segment, distribution channel and major product line:\nFISCAL 2023 COMPARED TO FISCAL 2022\n•NIKE, Inc. Revenues were $51.2 billion in fiscal 2023, which increased 10% and 16% compared to fiscal 2022 on a reported and currency-neutral basis, respectively.\nThe increase was due to higher revenues in North America, Europe, Middle East & Africa ("EMEA"), APLA and Greater China, which contributed approximately 7, 6,\n2 and 1 percentage points to NIKE, Inc. Revenues, respectively.\n•NIKE Brand revenues, which represented over 90% of NIKE, Inc. Revenues, increased 10% and 16% on a reported and currency-neutral basis, respectively. This\nincrease was primarily due to higher revenues in Men\'s, the Jordan Brand, Women\'s and Kids\' which grew 17%, 35%,11% and 10%, re

Словарь результатов содержит поля:

- `answer` с итоговым ответом.
- `context` с данными которые модель использовала для генерации ответа.

Поле `context` содержит информацию о документах, каждый из которых содержит фрагмент загруженного содержимого страницы.
Эти документы также сохраняют исходные метаданные с момента их первой загрузки:

In [9]:
print(results["context"][0].page_content)

Table of Contents
FISCAL 2023 NIKE BRAND REVENUE HIGHLIGHTS
The following tables present NIKE Brand revenues disaggregated by reportable operating segment, distribution channel and major product line:
FISCAL 2023 COMPARED TO FISCAL 2022
•NIKE, Inc. Revenues were $51.2 billion in fiscal 2023, which increased 10% and 16% compared to fiscal 2022 on a reported and currency-neutral basis, respectively.
The increase was due to higher revenues in North America, Europe, Middle East & Africa ("EMEA"), APLA and Greater China, which contributed approximately 7, 6,
2 and 1 percentage points to NIKE, Inc. Revenues, respectively.
•NIKE Brand revenues, which represented over 90% of NIKE, Inc. Revenues, increased 10% and 16% on a reported and currency-neutral basis, respectively. This
increase was primarily due to higher revenues in Men's, the Jordan Brand, Women's and Kids' which grew 17%, 35%,11% and 10%, respectively, on a wholesale
equivalent basis.


In [10]:
print(results["context"][0].metadata)

{'page': 35, 'source': '../example_data/nke-10k-2023.pdf'}


Фрагмент выше был взят с 35-й страницы оригинального PDF.
Вы можете использовать эти данные, чтобы показать, с какой страницы PDF был получен ответ. 
Это поможет пользователям быстро убедиться, что ответы основаны на исходном материале.

:::tip

Подробнее о RAG — в разделе [Создание RAG-приложения](/docs/tutorials/rag/) или в [руководствах](/docs/how_to/#qa-with-rag).

:::

## Смотрите также

- О загрузчиках документов в разделе [Основные понятия](/docs/concepts/#document-loaders)
- [Работа с загрузчиками документов](/docs/how_to/#document-loaders)
- [Доступные интеграции](/docs/integrations/document_loaders/)
- [Создание собственного загрузчика документов](/docs/how_to/document_loader_custom/)
- [Создание RAG-приложения](/docs/tutorials/rag/)
- [Руководства по работе с RAG](/docs/how_to/#qa-with-rag)